In [ ]:
import csv
import ast
import re
import numpy as np
import pandas as pd
import seaborn as sns

from Bio import SeqIO
from Bio.Seq import Seq

from matplotlib import pyplot as plt

In [ ]:
def align_read(querystr, cigarstr, pattern=re.compile("[0-9]{0,10}[MDI]")):
    result = pattern.finditer(cigarstr)
    cigar_seq = [(item.group(0)[-1], int(item.group(0)[:-1])) for item in result]
    #     output_str = "".join(["-" for i in range(cigar[1])])
    output_str = ""
    current_idx = 0
    for item in cigar_seq:
        if item[0] == "M":
            added_str = querystr[current_idx : current_idx + item[1]]
            output_str += added_str
            current_idx += item[1]
        elif item[0] == "D":
            added_str = "".join(["-" for i in range(item[1])])
            output_str += added_str
        elif item[0] == "I":
            current_idx += item[1]
    return output_str


def cigarsfromsam(samfilepath):
    cigars = {}
    with open(samfilepath, "r") as samfile:
        for line in samfile:
            if line[0] == "@":
                next(samfile)
            else:
                splitline = line.split("\t")
                cigars[splitline[0]] = splitline[5]
    return cigars


def strsfromfasta(fastafilepath):
    queries = SeqIO.to_dict(SeqIO.parse(fastafilepath, "fasta"))
    queries = {key: str(val.seq) for key, val in queries.items()}
    return queries


def make_seg_dict(gfafile):
    segment_dict = {}
    with open(gfafile, "r") as infile:
        for line in infile:
            if line[0] == "S":
                splitline = line.split("\t")
                segment_dict[splitline[1]] = splitline[2][:-1]
    return segment_dict


def get_ref_intervals(gfafile):
    segment_dict = {}
    current_idx = 0
    with open(gfafile, "r") as infile:
        for line in infile:
            if line[0] == "S":
                splitline = line.split("\t")
                if "OFF" not in splitline[1]:
                    refstr = splitline[2][:-1]
                    strlen = len(refstr)
                    name = splitline[1]
                    if "ON" in name:
                        name = name[:-2]
                    segment_dict[name] = tuple((current_idx, current_idx + strlen))
                    current_idx += strlen
    return segment_dict


def align_read(
    querystr, refstr, cigarstr, startpos=1, pattern=re.compile("[0-9]{0,10}[MDI]")
):
    start_pos = startpos - 1  ##comes as 1 indexed from minimap
    result = pattern.finditer(cigarstr)
    cigar_seq = [(item.group(0)[-1], int(item.group(0)[:-1])) for item in result]
    #     output_str = "".join(["-" for i in range(cigar[1])])
    output_str = ""
    if start_pos > 0:
        output_str += "".join(["-" for i in range(start_pos)])
    current_idx = 0
    for item in cigar_seq:
        if item[0] == "M":
            added_str = querystr[current_idx : current_idx + item[1]]
            output_str += added_str
            current_idx += item[1]
        elif item[0] == "D":
            added_str = "".join(["-" for i in range(item[1])])
            output_str += added_str
        elif item[0] == "I":
            current_idx += item[1]
    remaining_len = len(refstr) - len(output_str)
    if remaining_len > 0:
        output_str += "".join(["-" for i in range(remaining_len)])
    return output_str


def splitstr(instr, ref_intervals):
    strassign = {key: instr[val[0] : val[1]] for key, val in ref_intervals.items()}
    return strassign


def slow_hamming_distance(s1, s2):
    if len(s1) != len(s2):
        print(s1, s2)
        raise ValueError("Strand lengths are not equal!")
    term_list = []
    for ch1, ch2 in zip(s1, s2):
        if ch1 == "N" or ch2 == "N":
            term_list.append(False)
        else:
            term_list.append(ch1 != ch2)
    result = sum(term_list)
    return result


def get_dict_dist(dict1, dict2):
    hamming_dict = {
        key: slow_hamming_distance(dict1[key], dict2[key]) for key in dict1.keys()
    }
    return hamming_dict

In [ ]:
data = pd.read_csv(
    "/home/de64/scratch/de64/2020-11-23_lDE13_sequencing/output.tsv", delimiter="\t"
)
data["24bit_barcode"] = data["barcode"].apply(
    lambda x: x[:24]
)  # take off last 6 bits, since they were not recorded
ref_intervals = get_ref_intervals(
    "/home/de64/scratch/de64/2020-11-23_lDE13_sequencing/ref.gfa"
)
barcodes = set(data["barcode"].tolist())

In [ ]:
bit_arr = np.array([list(item) for item in barcodes]).astype(int)
bit_freq = np.mean(bit_arr, axis=0)

In [ ]:
len(bit_freq)

In [ ]:
fig = plt.figure(figsize=(16, 6))
sns.barplot(x=list(range(30)), y=bit_freq, color="grey")
plt.xlabel("Bit Number", fontsize=20)
plt.ylabel("Percent Positive", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.savefig("./figure_1.png", dpi=300, bbox_inches="tight")

In [ ]:
both_on = bit_arr @ bit_arr.T
both_off = (-bit_arr + 1) @ (-bit_arr.T + 1)
ttl_match = both_on + both_off
np.fill_diagonal(ttl_match, 100)

In [ ]:
closest_match = np.min(ttl_match, axis=0)

In [ ]:
plt.hist(closest_match, range=(0, 12), bins=9)
plt.xlabel("Closest Hamming Distance", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.savefig("./figure_2.png", dpi=300, bbox_inches="tight")

In [ ]:
plt.hist(closest_match, range=(0, 1), bins=2)
plt.xlabel("Closest Hamming Distance", fontsize=20)
plt.xticks([0.25, 0.75], [0, 1], fontsize=16)
plt.yticks(fontsize=16)
plt.savefig("./figure_3_24bits.png", dpi=300, bbox_inches="tight")
# plt.show()

In [ ]:
aligned_cons = data.apply(
    lambda x: align_read(
        x["consensus"], x["reference"], x["cigar"], startpos=x["alignmentstart"]
    ),
    axis=1,
)
data["aligned_cons"] = aligned_cons

split_ref = data.apply(lambda x: splitstr(x["reference"], ref_intervals), axis=1)
split_align = data.apply(lambda x: splitstr(x["aligned_cons"], ref_intervals), axis=1)
data["split_ref"] = split_ref
data["split_align"] = split_align

hamm_ref = data.apply(lambda x: get_dict_dist(x["split_align"], x["split_ref"]), axis=1)
data["hamm_ref"] = hamm_ref

data["sgRNA Target"] = data["split_align"].apply(lambda x: x["SGRNA"][175:195])

In [ ]:
data

In [ ]:
sgRNA_lib_df = pd.read_csv("./2020-11-01_mVenus_library_1.csv")
sgRNA_lib_df = sgRNA_lib_df.set_index("sequence")
sgRNA_lib_df = sgRNA_lib_df.rename(columns={"Unnamed: 0": "sgrnaid"})

In [ ]:
sgRNA_lib_df

In [ ]:
entry_list = []
for _, row in data.iterrows():
    try:
        entry = sgRNA_lib_df.loc[row["sgRNA Target"]]
        entry = pd.concat([row, entry])
        entry_list.append(entry)
    except:
        pass
output_df = pd.concat(entry_list, axis=1).T
del output_df["Unnamed: 0"]

In [ ]:
output_df.to_csv("./lDE13_final_df.tsv", sep="\t")

In [ ]:
output_df

In [ ]:
test = pd.read_csv("./lDE13_final_df.tsv", delimiter="\t", index_col=0)

In [ ]:
test